# Initial Data Cleaning
This script is designed to fill in nulls in important missing fields and to format and select the data which will be required to create a model later on. 

### Import Statements

In [1]:
import geopandas as gpd
import folium
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import sqlite3
from shapely.geometry import Point

## Filling in Nulls
To fill in nulls in this process we need some background on the data itself and how I intend to use it. First off, the dataset of interest contains over 2 million rows - each of which pertains to a specific wildfire. For each fire there is a Latitude and Longitude point for where the fire originated. Also, in most rows there are county and state code identifiers, however, in many rows (approx. 600K) there are no county identifiers. The lack of those county identifiers is an issue since we intend to analyze and predict wildfires on a county level. 

To address the issue of the missing county identifiers we take the following actions:

1. From an online source, get a data set which has Geofences (Polygons) for all the counties in the U.S. 
2. From the U.S. Census Bureau get a correlation of all the counties and states with their correct code 
3. Merge the county polygons to the county/state codes based on the county name and state abbreviation
4. In the wildfire data create spatial Points out of the longitude and latitude columns
5. Execute a Spatial merge between the wildfire Points and the county Polygons based on whether the Point is in the Polygon
6. Extract the county/state codes from the merged dataframe and then subset to the desired columns for further cleaning

Read in the .shp file which has all of the Polygons for the U.S. counties. Subset to just the three needed columns: county name, country/state/county abbreviation (which we extract state abbreviation from), and the Polygon for the county

In [2]:
geodf = gpd.read_file("./Data/CountyGeoFences/gadm36_USA_2.shp")[["NAME_2", "HASC_2", 'geometry']]
geodf.head(1)

,NAME_2,HASC_2,geometry
0,Autauga,US.AL.AU,"POLYGON ((-86.81896 32.34027, -86.81084 32.347..."


Create a new GeoDataFrame which reformats the county and state into the correct format to join on later

In [3]:
new_df = gpd.GeoDataFrame()
new_df['COUNTY'] = geodf['NAME_2'].str.upper().str.strip()
new_df['STATE'] = geodf['HASC_2'].str.slice(3,5).str.strip()
new_df['geometry'] = geodf['geometry']

new_df.head(1)

,COUNTY,STATE,geometry
0,AUTAUGA,AL,"POLYGON ((-86.81896 32.34027, -86.81084 32.347..."


Read in the Census data which correlates the county to the county/state code. In this data the *whole* states and the whole *US* are included, but are formatted so that their names are in all uppercase - whereas individual counties are in title case. So we filter out those rows which are already in upper case - as they are the whole states and whole US - which we are not interested in. 

In [4]:
lnd_data = pd.read_csv("./Data/CountyLandArea.csv")[['Areaname', 'STCOU']]
lnd_data = lnd_data.loc[lnd_data['Areaname'] != lnd_data['Areaname'].str.upper()]
lnd_data.head(1)

,Areaname,STCOU
2,"Autauga, AL",1001


Now format the data by separating the county name and the state abbreviation and by turning the code into a zero-padded five digit long value. For reference the code is built in a way such that the first two number indicate the state and the last three numbers indicate the county.

In [5]:
new_lnd_data = pd.DataFrame()
new_lnd_data['COUNTY'] = lnd_data['Areaname'].str.split(',').apply(lambda x: x[0].upper()).str.strip()
new_lnd_data['STATE'] = lnd_data['Areaname'].str.split(',').apply(lambda x: x[1].upper()).str.strip()
new_lnd_data['FIPS_CODE'] = lnd_data['STCOU'].astype(str).str.zfill(5)

new_lnd_data.head(1)

,COUNTY,STATE,FIPS_CODE
2,AUTAUGA,AL,01001


Merge the county Polygon data with the Census data to get the county/state code with the Polygons. Then ensure that the object is a Spatial DataFrame with the correct geospatial reference. 

In [6]:
comb = gpd.GeoDataFrame(pd.merge(new_lnd_data, new_df, on=['COUNTY', "STATE"], how='inner'), crs="EPSG:4326")

#### Set that dataset aside. 

Read in the wildfire data from the .sqlite table, selecting (in SQL) only the few columns which we want

In [7]:
cursor = sqlite3.connect('./Data/Fire/FPA_FOD_20210617.sqlite')
query = "SELECT f.FOD_ID, f.DISCOVERY_DATE, f.FIRE_SIZE, f.LATITUDE, f.LONGITUDE, f.FIPS_CODE from Fires f"
all_data = pd.read_sql_query(query, cursor)
all_data.head(1)

,FOD_ID,DISCOVERY_DATE,FIRE_SIZE,LATITUDE,LONGITUDE,FIPS_CODE
0,1,2/2/2005 0:00,0.1,40.036944,-121.005833,06063


Take the wildfire data and turn it into a Spatial DataFrame and create the geometry columns to be the Lat/Long points and ensure that the Spatial DataFrame has the correct reference

In [8]:
geo_fire_df = gpd.GeoDataFrame(all_data, geometry=gpd.points_from_xy(all_data.LONGITUDE, all_data.LATITUDE), crs="EPSG:4326")
geo_fire_df.head(1)

,FOD_ID,DISCOVERY_DATE,FIRE_SIZE,LATITUDE,LONGITUDE,FIPS_CODE,geometry
0,1,2/2/2005 0:00,0.1,40.036944,-121.005833,06063,POINT (-121.00583 40.03694)


Now take both DataFrames - with the points (wildfire data) and with the polygons (county/state data)

We will execute a spatial join such that we are joining the rows from the county/state data to the wildfire data wherever the Point in the wildfire data is *within* the Polygon of county/state data. 

In [9]:
merged = geo_fire_df.sjoin(comb, how='inner', predicate='within')

There may be some duplicates because of overlap in counties or some edge cases within the data. So we remove those duplicates and retain only the first instance of the row. 

In [10]:
mgd = merged.drop_duplicates(subset=["DISCOVERY_DATE", "FIRE_SIZE", "FIPS_CODE_right"])

## Formatting 

Now that we have the county/state codes filled in for all of our wildfire instances we can get to formatting the data. To format the data we will select just the few columns which we want and set the names correctly. Then we'll turn the discovery date into an actual date column and then finally calculate the discovery month of the fire - as we will be focused on monthly predictions of wildfires. 

In [11]:
final_mgd = mgd[["DISCOVERY_DATE", "FIRE_SIZE", "FIPS_CODE_right"]].rename({'FIPS_CODE_right': "FIPS_CODE"}, axis=1)
final_mgd.loc[:, 'DISCOVERY_DATE'] = pd.to_datetime(final_mgd.loc[:, 'DISCOVERY_DATE'])
final_mgd['MONTH'] = final_mgd['DISCOVERY_DATE'].to_numpy().astype('datetime64[M]')

final_mgd.head(1)

,DISCOVERY_DATE,FIRE_SIZE,FIPS_CODE,MONTH
0,2005-02-02,0.1,06063,2005-02-01


In [12]:
final_mgd.shape

(1857951, 4)

From this data we sum up all of the fire sizes for each month and county/state code (FIPS)

In [13]:
agg_data = final_mgd.groupby(by=['MONTH', 'FIPS_CODE']).agg({'FIRE_SIZE': sum}).reset_index()
agg_data.rename({'FIRE_SIZE': 'TOTAL_BURN_AREA'}, axis=1, inplace=True)

Now - we want to standardize the area burned by the size of the county itself. This way we can better assess the actual impact of the fires on the county since some small fires may be very impactful in certain counties (i.e. small ones) but large fires may not matter at all since it's really only a small portion of the county (i.e. in Alaska where the counties are larger than most states)

To do this standardization we turn back to the census bureau and we can find the land size (in Sq. Miles) for all of the counties and then merge it to our aggregated data and then divide the Fire Size by the county size. We will convert the Sq. Miles into Acres for a better apples-apples comparison of total area to area burned. 

Also note that some fires may extend beyond one county - or start in one county and then blow directly into the neighnoring county - and while this may lead some of our numbers to be decieving, as a whole the concept is still strong. However, from a data perspective this does mean that total area burned may be greater than county area. 

In [14]:
land_area_data = pd.read_csv("./Data/CountyLandArea.csv")[["STCOU", "LND010190D"]]
land_area_data.rename({'STCOU': "FIPS_CODE", "LND010190D": "COUNTY_AREA"}, axis=1, inplace=True)
land_area_data['FIPS_CODE'] = land_area_data["FIPS_CODE"].astype(str).str.zfill(5)
land_area_data['COUNTY_AREA'] = land_area_data['COUNTY_AREA'] * 640
land_area_data = land_area_data.iloc[np.where(land_area_data['COUNTY_AREA'] != 0)]

land_area_data.head(1)

,FIPS_CODE,COUNTY_AREA
0,00000,2.423952e+09


Now merge the aggregated data with the land area data and divide to get the burned to toal area ratio

In [15]:
agg_and_area = pd.merge(agg_data, land_area_data, how='inner', on=['FIPS_CODE'])
agg_and_area.head(1)

,MONTH,FIPS_CODE,TOTAL_BURN_AREA,COUNTY_AREA
0,1992-01-01,01015,8.0,391904.0


We will keep the burn area and county data so that we can aggregate separately for state and national geographic areas. So output the data. 

In [16]:
agg_and_area.to_csv("./Data/clean_fire_data.csv", index=False)

Our last step is to pull in some weather data, clean it and then join in based on the state for which the weather data pertains to. This data is kept separate for now but will be converted into a format where it can be easily joined into a state level aggregation of the wildfire data

First read in the weather data csv file

In [17]:
files = ["California", "Arizona", "Colorado", "Montana", "Nevada", "Oregon", "Washington", "Idaho", "Utah", "NewMexico", 
         "Wyoming"]
data = []
for f in files:
    df = pd.read_csv(f"./Data/Weather/{f}.csv", parse_dates=['DATE'])[["NAME", "DATE", "PRCP", "TMAX", "TMIN"]]
    df["STATE"] = df['NAME'].str.split(',').apply(lambda x: x[1]).str.strip().str.split(' ').apply(lambda x: x[0]).str.strip()
    df.drop('NAME', axis=1, inplace=True)
    data.append(df)
weather_data = pd.concat(data, axis=0)

Now, we will average out the measurements which we have for multiple stats down into measurements on a state level

In [18]:
grped_weather = weather_data.groupby(by=["STATE", "DATE"]).mean().reset_index()
grped_weather.rename({'DATE': 'MONTH'}, axis=1, inplace=True)

Now, once again, read in the Census data to get state codes - there is a bit of logic to extract the state codes but not too bad since we can use work we have already done to format the file correctly. In the end we can see that we do indeed end up with codes for our 50 states

In [19]:
census_data = pd.read_csv("./Data/CountyLandArea.csv")[["Areaname", "STCOU"]]
census_data = census_data.loc[census_data['Areaname'] != census_data['Areaname'].str.upper()]
census_data['STATE'] = census_data['Areaname'].str.split(',').apply(lambda x: x[1].upper()).str.strip()
census_data['STATE_CODE'] = census_data['STCOU'].astype(str).str.zfill(5).str.slice(0,2)
census_data = census_data.drop_duplicates(subset=['STATE_CODE', 'STATE'])[["STATE", "STATE_CODE"]].reset_index(drop=True)

Now we will join the weather and census data together to get the state codes for our states

In [20]:
weather_w_codes = pd.merge(grped_weather, census_data, how='inner', on='STATE')

Now export the data to a csv file where we can pull it into our modelling efforts later on

In [21]:
weather_w_codes.to_csv("./Data/cleaned_weather_data.csv", index=False)